### Stable Marriage Problem

This algorithm was developed by David Gayle and Lloyd Shapely in 1962.

In [81]:
women = {"Juliette":["Romeo", "Baxter", "Frank", "Leo", "Asher"],
         "Maeve":["Baxter", "Leo", "Asher", "Romeo", "Frank"],
         "Sadie":["Baxter", "Asher", "Leo", "Frank", "Romeo"],
         "Libby": ["Leo", "Frank","Romeo","Asher","Baxter"],
         "Bianca":["Asher", "Baxter", "Frank", "Romeo","Leo"]}

In [82]:
men = {"Romeo":["Libby", "Maeve","Sadie","Bianca", "Juliette"],
       "Baxter":["Juliette", "Maeve", "Bianca", "Sadie","Libby"],
       "Asher": ["Sadie", "Libby", "Bianca", "Juliette", "Maeve"],
       "Frank": ["Juliette", "Bianca","Libby","Maeve","Bianca"],
       "Leo":["Juliette", "Libby", "Bianca", "Sadie","Maeve"]}

In [88]:
def matchmaker(suitor_picks, reviewer_picks):
    suitors = list(suitor_picks.keys())
    matched = {s: None for s in suitors}
    
    while len(suitors) > 0:
        s = suitors.pop(0)
        r = suitor_picks[s][0]
        if r not in matched.values():
            matched[s] = r
        else:
            for suitor, reviewer in matched.items():
                if reviewer == r:
                    r_partner = suitor
            if reviewer_picks[r].index(s) < reviewer_picks[r].index(r_partner):
                matched[r_partner] = None
                matched[s] = r
                suitors.append(r_partner)
            else:
                suitor_picks[s].remove(r)
                suitors.append(s)
    return matched

In [89]:
matchmaker(men,women)

{'Romeo': 'Maeve',
 'Baxter': 'Juliette',
 'Asher': 'Sadie',
 'Frank': 'Bianca',
 'Leo': 'Libby'}

In [90]:
matchmaker(women,men)

{'Juliette': 'Romeo',
 'Maeve': 'Baxter',
 'Sadie': 'Asher',
 'Libby': 'Leo',
 'Bianca': 'Frank'}